# MSWT Anwendung in der Finanzbranche 
In diesem Notebook wird die Anwendung des selben Prinzips/Logik wie in der Filmbranche demonstriert

## Neo4j Verbindung erstellen

In [ ]:
from neo4j import GraphDatabase
uri = "bolt://localhost:7687"  
user = "neo4j"  
password = "" 

driver = GraphDatabase.driver(uri, auth=(user, password))

## Daten Erstellen 

In [ ]:
from neo4j import GraphDatabase

class FinancialGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def setup_graph(self):
        with self.driver.session() as session:
            # Create accounts
            session.run("CREATE (:Account {id: 'acc1', name: 'Account 1'})")
            session.run("CREATE (:Account {id: 'acc2', name: 'Account 2'})")
            session.run("CREATE (:Account {id: 'acc3', name: 'Account 3'})")
            session.run("CREATE (:Account {id: 'acc4', name: 'Account 4'})")

            # Create transactions
            session.run("""
            MATCH (a1:Account {id: 'acc1'}), (a2:Account {id: 'acc2'})
            CREATE (a1)-[:TRANSACTION {amount: 1000}]->(a2)
            """)
            session.run("""
            MATCH (a1:Account {id: 'acc2'}), (a2:Account {id: 'acc3'})
            CREATE (a1)-[:TRANSACTION {amount: 500}]->(a2)
            """)
            session.run("""
            MATCH (a1:Account {id: 'acc3'}), (a2:Account {id: 'acc4'})
            CREATE (a1)-[:TRANSACTION {amount: 200}]->(a2)
            """)
            session.run("""
            MATCH (a1:Account {id: 'acc4'}), (a2:Account {id: 'acc1'})
            CREATE (a1)-[:TRANSACTION {amount: 800}]->(a2)
            """)
            session.run("""
            MATCH (a1:Account {id: 'acc1'}), (a2:Account {id: 'acc3'})
            CREATE (a1)-[:TRANSACTION {amount: 700}]->(a2)
            """)
            session.run("""
            MATCH (a1:Account {id: 'acc2'}), (a2:Account {id: 'acc4'})
            CREATE (a1)-[:TRANSACTION {amount: 300}]->(a2)
            """)

## Function to create graph projection

In [ ]:
def create_graph_projection(tx):
    query = """
    CALL gds.graph.project(
        'graphAcc2',
        'Scene',
        {
            RELATED: {
                properties: 'amount',
                orientation: 'UNDIRECTED'
            }
        }
    )
    """
    tx.run(query)

## Function to compute the minimum spanning tree and write it to the database

In [ ]:
def compute_spanning_tree(tx, account_id):
    query = """
    MATCH (n:Account {id: $account_id})
    CALL gds.spanningTree.write('graphAcc2', {
        sourceNode: n,
        relationshipWeightProperty: 'amount',
        writeProperty: 'writeAmount',
        writeRelationshipType: 'MINST'
    })
    YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    """
    result = tx.run(query, account_id=account_id)
    return result.single()

## Function to fetch the spanning tree relationships

In [ ]:
def fetch_spanning_tree_relationships(tx, account_id):
    query = """
    MATCH path = (n:Account {id: $account_id})-[:MINST*]-()
    WITH relationships(path) AS rels
    UNWIND rels AS rel
    WITH DISTINCT rel AS rel
    RETURN startNode(rel).id AS Source, endNode(rel).id AS Destination, rel.writeAmount AS Weight
    """
    result = tx.run(query, account_id=account_id)
    return result.data()

## Function to perform DFS

In [ ]:
def perform_dfs(tx, account_id):
    query = """
    MATCH (start:Account {id: $account_id})
    CALL gds.dfs.stream('graphAcc', {
        sourceNode: id(start)
    })
    YIELD path
    UNWIND nodes(path) AS node
    RETURN collect(node.id) AS visitOrder
    """
    result = tx.run(query, account_id=account_id)
    return result.single()['visitOrder']

## Function to perform BFS

In [ ]:
def perform_bfs(tx, account_id):
    query = """
    MATCH (start:Account {id: $account_id})
    CALL gds.bfs.stream('graphAcc', {
        sourceNode: id(start)
    })
    YIELD path
    UNWIND nodes(path) AS node
    RETURN collect(node.id) AS visitOrder
    """
    result = tx.run(query, account_id=account_id)
    return result.single()['visitOrder']

## Run all functions

In [ ]:
with driver.session() as session:
    # Create graph projection
    session.write_transaction(create_graph_projection)
    
    # Account ID
    account_id = 'acc1'
    
    # Compute spanning tree
    spanning_tree_result = session.write_transaction(compute_spanning_tree, account_id)
    print("Spanning Tree Result:", spanning_tree_result)
    
    # Fetch spanning tree relationships
    spanning_tree_relationships = session.read_transaction(fetch_spanning_tree_relationships, account_id)
    print("Spanning Tree Relationships:", spanning_tree_relationships)
    
    # Perform DFS
    dfs_result = session.read_transaction(perform_dfs, account_id)
    print("DFS Visit Order:", dfs_result)
    
    # Perform BFS
    bfs_result = session.read_transaction(perform_bfs, account_id)
    print("BFS Visit Order:", bfs_result)

driver.close()

## Karusselbetrug mit MWST

In [ ]:
from neo4j import GraphDatabase

class FinancialExample:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_example_data(self):
        with self.driver.session() as session:
            session.run(
                """
                // Personen erstellen
                CREATE (p1:Person {name: 'Alice', id: '1'})
                CREATE (p2:Person {name: 'Bob', id: '2'})
                CREATE (p3:Person {name: 'Charlie', id: '3'})
                CREATE (p4:Person {name: 'Dave', id: '4'})
                CREATE (p5:Person {name: 'Eve', id: '5'})
                
                // Konten erstellen
                CREATE (a1:Account {accountNumber: 'A1', balance: 10000})
                CREATE (a2:Account {accountNumber: 'A2', balance: 15000})
                CREATE (a3:Account {accountNumber: 'A3', balance: 30000})
                CREATE (a4:Account {accountNumber: 'A4', balance: 5000})
                CREATE (a5:Account {accountNumber: 'A5', balance: 7000})
                
                // Beziehungen zwischen Personen und Konten erstellen
                CREATE (p1)-[:OWNS]->(a1)
                CREATE (p1)-[:OWNS]->(a2)
                CREATE (p2)-[:OWNS]->(a3)
                CREATE (p3)-[:OWNS]->(a4)
                CREATE (p4)-[:OWNS]->(a5)
                
                // Transaktionen erstellen
                CREATE (a1)-[:TRANSFERRED {amount: 7000, date: '2023-01-01'}]->(a2)
                CREATE (a2)-[:TRANSFERRED {amount: 5000, date: '2023-01-02'}]->(a3)
                CREATE (a3)-[:TRANSFERRED {amount: 30000, date: '2023-01-03'}]->(a4)
                CREATE (a4)-[:TRANSFERRED {amount: 7000, date: '2023-01-05'}]->(a5)
                CREATE (a5)-[:TRANSFERRED {amount: 7000, date: '2023-01-06'}]->(a1)
                CREATE (a4)-[:TRANSFERRED {amount: 5000, date: '2023-01-04'}]->(a1)
                """
            )

    def project_graph(self):
        with self.driver.session() as session:
            session.run(
                """
                    CALL gds.graph.project(
                        'graphAcc20',
                        'Account',
                        {
                            TRANSFERRED: {
                                properties: 'amount',
                                orientation: 'UNDIRECTED'
                            }
                        }
                    )
                
                """
            )

    def calculate_spanning_tree(self):
        with self.driver.session() as session:
            result = session.run(
                """
                MATCH (n:Account {accountNumber: 'A1'})
                CALL gds.spanningTree.write('graphAcc20', {
                  sourceNode: n,
                  relationshipWeightProperty: 'amount',
                  writeProperty: 'writeAmount',
                  writeRelationshipType: 'MINST'
                })
                YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
                RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
                """
            )
            for record in result:
                print("Spanning Tree Calculation:")
                print(f"PreProcessingMillis: {record['preProcessingMillis']}")
                print(f"ComputeMillis: {record['computeMillis']}")
                print(f"WriteMillis: {record['writeMillis']}")
                print(f"EffectiveNodeCount: {record['effectiveNodeCount']}")

    def find_cycles(self):
        with self.driver.session() as session:
            result = session.run(
                """
                MATCH path = (n:Account {accountNumber: 'A1'})-[:TRANSFERRED*]->(n)
                WITH relationships(path) AS rels
                UNWIND rels AS rel
                WITH DISTINCT rel AS rel
                RETURN startNode(rel).accountNumber AS Source, endNode(rel).accountNumber AS Destination, rel.amount AS Weight
                """
            )
            print("Cycles in Transactions:")
            for record in result:
                print(f"Source: {record['Source']}, Destination: {record['Destination']}, Weight: {record['Weight']}")

if __name__ == "__main__":
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "" 

    example = FinancialExample(uri, user, password)

    # Beispiel-Daten erstellen
    print("Erstellen der Beispiel-Daten...")
    example.create_example_data()

    # Graph projizieren
    print("Projizieren des Graphen...")
    example.project_graph()

    # Spanning Tree berechnen
    print("Berechnen des Spanning Tree...")
    example.calculate_spanning_tree()

    # Zyklen in Transaktionen finden
    print("Zyklen in Transaktionen finden...")
    example.find_cycles()

    example.close()


## Übergabe an GPT

In [ ]:
import openai

class OpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def generate_chat_response(self, prompt):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in fraud detection and money laundering investigation and you have to decide if the following transactions are money laundering. Please analyze the following transaction data and provide a conlcusion."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message['content']

# Funktion zur Analyse der Transaktionen und Generierung des Prompts
def analyze_transactions(api_key, transactions):
    openai_wrapper = OpenAIWrapper(api_key)
    
    prompt = "Cycles in Transactions:\n"
    for transaction in transactions:
        prompt += f"Source: {transaction['Source']}, Destination: {transaction['Destination']}, Weight: {transaction['Weight']}\n"

    prompt += "\nIs this a suspicious pattern indicating potential money laundering? How should I decide if this is a case of potential money laundering?"

    response = openai_wrapper.generate_chat_response(prompt)
    return response

transactions = [
    {"Source": "A1", "Destination": "A2", "Weight": 7000},
    {"Source": "A2", "Destination": "A3", "Weight": 5000},
    {"Source": "A3", "Destination": "A4", "Weight": 30000},
    {"Source": "A4", "Destination": "A5", "Weight": 7000},
    {"Source": "A5", "Destination": "A1", "Weight": 7000},
    {"Source": "A4", "Destination": "A1", "Weight": 5000}
]


api_key = ''

# Analyse der Transaktionen und Abrufen der GPT-3 Antwort
response = analyze_transactions(api_key, transactions)
print(response)
